In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 42.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=31f71f1ac283662b76b74e66be4cbbd4acd6049cb54bf9a0375a03e32cfcc5c1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, DateType,FloatType
from pyspark.sql.functions import col, asc,desc,lit, sqrt,stddev, mean, udf
from collections import Counter
import json
import csv
from io import StringIO
from pyspark.sql import HiveContext
from sklearn import datasets 
import pandas as pd


In [3]:


sc = SparkContext()
spark = SparkSession(sc)


In [4]:
iris = datasets.load_iris()
iris

{'DESCR': '.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n                \n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n    ===========

In [5]:
type(iris)

sklearn.utils.Bunch

In [6]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
iris_df['target'] = [str(iris.target_names[x]) for x in iris.target]
# iris_df['target'] =  iris.target

In [8]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [9]:
iris_df.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target                object
dtype: object

Write a function that reads the data using Spark data frames. Converting Pandas Dataframe to Spark Dataframe

In [10]:
iris_spark_df = spark.createDataFrame(iris_df)


In [11]:
iris_spark_df.printSchema()


root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- target: string (nullable = true)



In [12]:
iris_spark_df.show()

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|setosa|
|              4.9|             3.0|              1.4|             0.2|setosa|
|              4.7|             3.2|              1.3|             0.2|setosa|
|              4.6|             3.1|              1.5|             0.2|setosa|
|              5.0|             3.6|              1.4|             0.2|setosa|
|              5.4|             3.9|              1.7|             0.4|setosa|
|              4.6|             3.4|              1.4|             0.3|setosa|
|              5.0|             3.4|              1.5|             0.2|setosa|
|              4.4|             2.9|              1.4|             0.2|setosa|
|              4.9|             3.1|              1.

Normalize the data:
○ Subtract the mean of the respective column from each cell, and divide that with
the standard deviation of that column. Check the below url on how to do that:
○ Mean: https://sparkbyexamples.com/spark/spark-sql-aggregate-functions/#mean
○ Standard Deviation:
https://sparkbyexamples.com/spark/spark-sql-aggregate-functions/#stddev

In [13]:
def standarzise_data(iris_spark_df):
  column = iris_spark_df.columns[:-1]
  for col_name in column:
    col_mean, col_sd = iris_spark_df.select(mean(col_name), stddev(col_name)).first()
    iris_spark_df = iris_spark_df.withColumn(col_name, (iris_spark_df[col_name] - col_mean)/col_sd )

  return iris_spark_df


In [14]:
iris_spark_df = standarzise_data(iris_spark_df)

In [15]:
iris_spark_df.show()

+--------------------+--------------------+-------------------+-------------------+------+
|   sepal length (cm)|    sepal width (cm)|  petal length (cm)|   petal width (cm)|target|
+--------------------+--------------------+-------------------+-------------------+------+
| -0.8976738791967641|   1.015601990713633|-1.3357516342415203|-1.3110521482051298|setosa|
| -1.1392004834649514| -0.1315388120502606|-1.3357516342415203|-1.3110521482051298|setosa|
| -1.3807270877331395|  0.3273175090552973|-1.3923992862449777|-1.3110521482051298|setosa|
| -1.5014903898672343| 0.09788934850251833|-1.2791039822380632|-1.3110521482051298|setosa|
| -1.0184371813308577|   1.245030151266412|-1.3357516342415203|-1.3110521482051298|setosa|
| -0.5353839727944812|  1.9333146329247477|-1.1658086782311488|-1.0486667949952972|setosa|
| -1.5014903898672343|  0.7861738301608542|-1.3357516342415203|-1.1798594716002135|setosa|
| -1.0184371813308577|  0.7861738301608542|-1.2791039822380632|-1.3110521482051298|setosa|

In [16]:
iris_spark_df.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'target']

Randomly divide the data into three sets such that one set has around 60% data sample
and the other two have 20% data sample each. Let's call the 60% set as the train set
and the 20% set as the validation set and the other 20% as the test set. (Note: This has
to be a random division)

In [41]:
train_df, val_df, test_df = iris_spark_df.randomSplit([0.6, 0.2, 0.2], seed = 12)

In [42]:
train_df.count(), val_df.count(), test_df.count()

(102, 19, 29)

In [43]:
train_df.show()

+-------------------+-------------------+-------------------+--------------------+----------+
|  sepal length (cm)|   sepal width (cm)|  petal length (cm)|    petal width (cm)|    target|
+-------------------+-------------------+-------------------+--------------------+----------+
| -1.863780296269516|-0.1315388120502606|-1.5056945902518917|  -1.442244824810046|    setosa|
|-1.7430169941354212| 0.3273175090552973|-1.3923992862449777| -1.3110521482051298|    setosa|
|-1.6222536920013277|-1.7375359359197122|-1.3923992862449777| -1.1798594716002135|    setosa|
|-1.5014903898672343|0.09788934850251833|-1.2791039822380632| -1.3110521482051298|    setosa|
|-1.5014903898672343| 0.3273175090552973|-1.3357516342415203| -1.3110521482051298|    setosa|
|-1.5014903898672343|  1.245030151266412| -1.562342242255349| -1.3110521482051298|    setosa|
|-1.3807270877331395| 0.3273175090552973|-1.3923992862449777| -1.3110521482051298|    setosa|
|-1.3807270877331395| 0.3273175090552973|-1.2224563302346059

In [44]:
val_df.show()

+--------------------+-------------------+-------------------+--------------------+----------+
|   sepal length (cm)|   sepal width (cm)|  petal length (cm)|    petal width (cm)|    target|
+--------------------+-------------------+-------------------+--------------------+----------+
| -1.7430169941354212|-0.1315388120502606|-1.3923992862449777| -1.3110521482051298|    setosa|
|  -1.259963785599046|-0.1315388120502606|-1.3357516342415203| -1.1798594716002135|    setosa|
| -1.1392004834649514|0.09788934850251833|-1.2791039822380632|  -1.442244824810046|    setosa|
| -1.0184371813308577| -2.425820417578048|  -0.14615094216892|-0.26151073536579955|versicolor|
| -1.0184371813308577| 0.7861738301608542|-1.2791039822380632| -1.3110521482051298|    setosa|
| -0.7769105770626695|-0.8198232937085964|0.08043966584490861| 0.26325997105386545|versicolor|
| -0.1730940663921995| 1.7038864723719689|-1.1658086782311488| -1.1798594716002135|    setosa|
| -0.5353839727944812|-0.1315388120502606| 0.42032

In [45]:
test_df.show()

+--------------------+--------------------+-------------------+--------------------+----------+
|   sepal length (cm)|    sepal width (cm)|  petal length (cm)|    petal width (cm)|    target|
+--------------------+--------------------+-------------------+--------------------+----------+
| -1.7430169941354212|-0.36096697260303956|-1.3357516342415203| -1.3110521482051298|    setosa|
| -1.5014903898672343|  0.7861738301608542|-1.3357516342415203| -1.1798594716002135|    setosa|
| -1.1392004834649514| 0.09788934850251833|-1.2791039822380632| -1.3110521482051298|    setosa|
| -1.0184371813308577|  0.5567456696080751|-1.3357516342415203| -1.3110521482051298|    setosa|
| -1.0184371813308577|  0.7861738301608542|-1.2224563302346059| -1.0486667949952972|    setosa|
| -1.0184371813308577|   1.015601990713633|-1.3923992862449777| -1.1798594716002135|    setosa|
| -0.8976738791967641|   1.015601990713633|-1.3357516342415203| -1.1798594716002135|    setosa|
| -0.7769105770626695|  0.78617383016085

● Write a function which calculates the Euclidean Distance between two rows:
https://en.wikipedia.org/wiki/Euclidean_distance
○ Note: the class column is ignored while calculating the distance.
○ Optional: You can write this function in such a way that you can calculate the
distance of one row with all the other rows at once.

In [83]:

def euclideanDistance(data_1, data_2): 
    euc_distance = 0
   
    cntr = 0
    while cntr < len(data_1)-1:
          euc_distance = euc_distance + pow((data_1[cntr] - data_2[cntr]), 2)
          cntr += 1
    
    # print(dist)

    return (euc_distance)**0.5



    

Fetching k nearest neighbor: Randomly guess a value for k, let’s say k = 5. For each
row in the validation data set, repeatedly use your euclidean function to calculate the
distance with each row of the train set. Once you have all the distances, select the k
minimum distances and get the corresponding training data samples for these k
distances.


● Making a prediction: From the above k train samples, get the most frequent class label.
This class label is the prediction for one validation data sample/row. Repeat the above
two points for each data sample in the validation data sample/row.

In [144]:
def getNeighbours(K, val_df, train_df):
   
    all_distance_target_list = []
    for i in range(train_df.count()):
        euc_distiance = euclideanDistance(train_df.collect()[i], val_df)
        
        all_distance_target_list.append([euc_distiance, train_df.collect()[i][len(val_df)-1]])
        all_distance_target_list.sort()
        
    
    neigbours_list = [x[1] for x in all_distance_target_list[:K]]   

    return neigbours_list

In [146]:
K = [5, 10, 15, 20, 25]

In [149]:
def get_predictions(k, val_df, train_df):
      targets_list = getNeighbours(k, val_df, train_df)
      # print(targets_list)
      counts_of_all_targets = Counter(targets_list)

      final_target = None
      final_count = 0

      for target, count in counts_of_all_targets.items():
        if count > final_count:
          final_count = count
          final_target = target

      return final_target


Training

In [153]:
K_training_accuracies = []
for k in range(len(K)):
    # acc, _, _ = get_accuracy(train_df, val_df, K[k])  acc / test_df.count(), preds, actuals

    count_of_accuracy = 0
    predictions = []
    true_values = []
    for i in range(val_df.count()):
        pred = get_predictions(k, val_df.collect()[i], train_df)
        true_val = val_df.collect()[i][len(val_df.collect()[0])-1]

        if pred == true_val:
            count_of_accuracy += 1
        predictions.append(pred)
        true_values.append(true_val)
    

    K_training_accuracies.append(count_of_accuracy/val_df.count())

In [159]:

for i in range(len(K)):
    print("Training/Validation accuracy is ", K_training_accuracies[i], " for k ", K[i])

Training/Validation accuracy is  0.0  for k  5
Training/Validation accuracy is  1.0  for k  10
Training/Validation accuracy is  1.0  for k  15
Training/Validation accuracy is  1.0  for k  20
Training/Validation accuracy is  1.0  for k  25


In [156]:
best_k = K[K_training_accuracies.index(max(K_training_accuracies))]

In [157]:
best_k

10

Testing

In [160]:

count_of_accuracy = 0
predictions = []
true_values = []
for i in range(test_df.count()):
    pred = get_predictions(k, test_df.collect()[i], train_df)
    true_val = test_df.collect()[i][len(test_df.collect()[0])-1]
    if pred == true_val:
        count_of_accuracy += 1
    predictions.append(pred)
    true_values.append(true_val)


print("Predicted Values: ", predictions)
print("Actual Values: ", true_values)
print("Accuracy: ", count_of_accuracy/test_df.count())

Predicted Values:  ['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica']
Actual Values:  ['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica']
Accuracy:  0.9310344827586207


The Testing Accuracy is 93.10%

In [178]:

euclideanDistance(train_df.collect()[0], train_df.collect()[1])


0.48782038136067724

References:

https://en.wikipedia.org/wiki/Iris_flower_data_set#Use_of_the_data_set

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html


https://medium.com/where-quant-meets-data-science/building-k-nearest-neighbour-algorithm-from-scratch-bd0c5df13192

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html